In [2]:
import pandas as pd
ministers = pd.read_csv('./ministers.csv', encoding="utf-16")
ministers.head()

,name,sex,age,country
0,Karen Karapetyan,M,53,Armenia
1,Vigen Sargsyan,M,42,Armenia
2,Vache Gabrielyan,M,48,Armenia
3,Suren Karayan,M,44,Armenia
4,Ignati Arakelyan,M,45,Armenia


In [16]:
def filter_sex(sex: str):
    only_sex = ministers[ministers['sex'] == sex]
    return only_sex.groupby('country').count()['sex']
    
only_men = filter_sex('M')
only_women = filter_sex('F')

sex_ratio = pd.DataFrame({'men': only_men, 'women': only_women}).fillna(0)
sex_ratio['women'] = sex_ratio['women'].astype(int)
sex_ratio['total'] = sex_ratio['men'] + sex_ratio['women']

sex_ratio['ratio'] = sex_ratio['men'] / sex_ratio['total']
sex_ratio.head()


,men,women,total,ratio
country,,,,
Armenia,18,1,19,0.947368
Azerbaijan,26,0,26,1.000000
Belarus,26,2,28,0.928571
Estonia,11,4,15,0.733333
Georgia,17,1,18,0.944444
